### IMPORTS

In [136]:
import os
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np



### HELPER FUNCTIONS

In [137]:
def get_subdirectories(folder_path):
    subdirectory_path_list = []
    subdirectory_names = os.listdir(folder_path)

    for subdirectories in subdirectory_names:
        full_path = os.path.join(folder_path, subdirectories)
        subdirectory_path_list.append(full_path)
    
    return subdirectory_path_list

def get_image(image_file):
    # Load the image
    image = Image.open(image_file)
    # Convert the image to a NumPy array
    image_array = np.array(image)
    return image_array
    

def preprocess_dataset(class_files):
    label_list  = []

    for i in range(len(class_files)):
        subdirectories = get_subdirectories(class_files[i])
        subdirectories.sort()

        class_files[i] = subdirectories
        
        for sub in subdirectories: 
            label_list.append(i)

    #Combining the classes together
    concated_list = []
    for classes in class_files:
        concated_list.extend(classes)

    #Shuffling the classes
    concated_list = np.array(concated_list)
    label_list = np.array(label_list)

    concated_list.sort()
    
    print(concated_list[1])
    print(label_list[1])

    # Generate a random permutation of indices
    indices = np.random.permutation(len(concated_list))

    # Apply the permutation to both lists
    shuffled_class_list = concated_list[indices]
    shuffled_label_list = label_list[indices]
    
    # for image_file in shuffled_class_list: 
    #     image_file = get_image(image_file)

    return shuffled_class_list, shuffled_label_list

In [138]:
path = kagglehub.dataset_download("tawsifurrahman/covid19-radiography-database")
files = os.listdir(path)
print("Path to files", path)
print(files)

Path to files /home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5
['COVID-19_Radiography_Dataset']


In [139]:
main_folder = os.path.join(path, files[0])
files_list = get_subdirectories(main_folder)
print(np.array(files_list))

['/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/Viral Pneumonia'
 '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/COVID'
 '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/Viral Pneumonia.metadata.xlsx'
 '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/Lung_Opacity.metadata.xlsx'
 '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/COVID.metadata.xlsx'
 '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/Normal'
 '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/Lung_O

In [140]:
#Getting the files from the dataset
normal_file = files_list[5]
normal_subdirectories = get_subdirectories(normal_file)
print(normal_subdirectories)

covid_file = files_list[1]
coivd_subdirectories = get_subdirectories(covid_file)
print(coivd_subdirectories)

covid_images = coivd_subdirectories[1]
covid_mask = coivd_subdirectories[0]

normal_images = normal_subdirectories[1]
normal_mask = normal_subdirectories[0]

['/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/Normal/masks', '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/Normal/images']
['/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/COVID/masks', '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/COVID/images']


In [141]:
class_files = [covid_images, normal_images]
class_files, label_list = preprocess_dataset(class_files)

print(len(label_list))
print(len(class_files))

/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/COVID/images/COVID-10.png
0
13808
13808


In [142]:

# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42)